In [4]:
import pandas as pd
import pyodbc 
import sqlalchemy
import time
import os

driver = 'ODBC Driver 17 for SQL Server'
server = 'ig-seau-dev-reportingau-data.database.windows.net' 
database = 'AI' 
username = 'data_teamau' 
password = 'Tu5harTab!' 

In [5]:
path = 'G:/Reporting & Insights/Live/02 Internal - New Claims Summary/'
os.chdir(path)
CFT_files = [f for f in os.listdir(path) if f.startswith('tableau')]
CFT_df_list = list()
for file in CFT_files:
    print(f'----------- Processing {file} --------------------------' )
    CFT_df_list.append(pd.read_csv(file))

CFT_df_union = pd.concat(CFT_df_list,ignore_index=True,axis=0)
    

----------- Processing tableau_summary_p1.csv --------------------------


C:\Users\LiaoG\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (4,26,30,31,32,33,34,35,36,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


----------- Processing tableau_summary_p2.csv --------------------------


C:\Users\LiaoG\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (4,34) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


----------- Processing tableau_summary_p3.csv --------------------------


In [6]:
def replacePolicyType(x):
    if x.startswith('AZ') or x in ['Coverforce Novated Lease','Dawes']:
        return 'Allianz'
    
    elif x.strip() in ['Steadfast Commercial Motor Vehicle Insurance']:
        return 'Allied World'
    
    elif x.strip().startswith('Home and Contents'):
        return 'BH IAL Property'
    
    elif x.strip() in ['BH Landlords','IAL Landlords','Landlord Accidental Damage','Landlord Listed Events']:
        return 'BH Landlords'
    
    elif x.strip().endswith('(Calliden)'):
        return 'Calliden - SUA'
    
    elif x.strip().strip() in ['Affinity Risk Partners',
    			'Affinity Risk Recovery',
    			'Australian Reliance',
    			'Bluedog Motor',
    			'Commercial - QUS',
              'Commercial Motor Vehicle Insurance',
              'CRISP Motor',
               'Dawes Dealership',
               'Dawes Famous',
               'Dawes General',
               'Dawes Learner Driver (CADL)',
               'Dawes Marque',
               'Dawes Young Driver',
                'E-Bike',
                'E-Car',
                'Famous Classic',
                'Famous Learner Driver',
                'Famous Motor',
                'Famous TPPD Only',
                'Freeman Mcmurrick',
                'Heritage Classic Policy',
                'Heritage Learner Driver Policy',
                'Heritage Motor Policy',
                'Motor - Calliden',
                'motor - c',
                'Insure Simply',
                'Mybike - 40 Plus',
                'Mybike - Bike',
                'Mydirtbike',
                'Myscooter',
                'NLC Lease Guarantee',
                'NLC Motor',
                'Recreational Vehicles',
                'Remunerator Car Policy Scheme',
                'Ryno Insurance Bike',
                'Ryno Insurance Car',
                'Taxi Scheme',
                'Trucksure Aggregate',
                'Trucksure Plus',
                'Trucksure Standard',
                'Young & Cool']:
        return 'Calliden Insurance Limited'
    
    elif x.strip() in ['Calliden Commercial']:
        return 'Calliden Warranty'
    
    elif x.strip() in ['Calliden Landlords','CIL Steadfast Landlords','ACE Mansions - Building And Contents','NSW Trustees & Guardian','Watkins Taylor Stone - Landlords']:
        return 'Chubb Insurance'
    
    elif x.strip() in['Builders Warranty',
                'Builders Warranty VIC',
                'Builders Warranty WA',
                'NSW Warranty',
            'Owner Builder']:
        return 'CIL - Builders Warranty'
    
    elif x.strip() in ['WA Builders Warranty (GLA)']:
        return 'CIL - Builders Warranty WA'
    
    elif x.strip() in ['ABF - Elite Care',
            'Argis - Home Property',
            'Argis - Pensioner Property',
            'Argis Elite Care',
            'Argis Elite Care Plus',
            'Argis Elite Care Steadfast',
            'Argis Extra Care',
            'Argis Extra Care plus',
            'Argis Extra Care Steadfast',
            'Argis Landlords',
            'Calliden Elite Care',
            'Calliden Elite Care Plus',
            'Calliden Everyday Care',
            'Calliden Extra Care',
            'Calliden Extra Care Plus',
            'Calliden Landlords - Old',
            'CIL Steadfast Landlords - Old',
            'IBNA Gold',
            'IBNA Silver',
            'Steadfast Elite Care',
            'Steadfast Extra Care',
            'Unisys Run Off']:
        return 'CIL - Domestic'
    
    elif x.strip() in ['SUA - Australian Calisthenics Fed',
                'SUA - Australian Flying Disc Assoc',
                'SUA - Australian Oztag Assoc',
                'SUA - Brisbane City Indoor Sports',
                'SUA - Gymnastics Australia',
                'SUA - Netball Australia',
                'SUA - Other',
                'SUA - Touch Rugby League']:
        return 'CIL - SUA' 
    
    elif x.strip() in ['Argis - Commercial Motor',
                'Argis - Farm Motor',
                'Argis - Home Motor',
                'CIL Taxi Scheme Recovery',
                'CIL Taxi Scheme Standard',
                'CRISP Commercial Motor']:
        return 'CIL Motor'

    elif x.strip() in ['Famous Car',
                'Famous My 40',
                'Famous My Bike',
                'Famous My Dirt',
                'Famous My Scooter']:
        return 'Famous Motor'    
    
    elif x.strip().endswith('(GLA)'):
        return 'GLA - SUA'

    elif x.strip() in ['Dawes Dirt bike',
                'Dawes Driving Instructor',
                'Dawes Motor Vehicle',
                'Dawes Motorcycle',
                'Dawes Porsche',
                'GLA Australian Reliance']:
        return 'GLA Dawes Motor'
            
    
    
    
    elif x.strip().startswith('GLA Famous') or x in ['RAC Classic']:
        return 'GLA Famous'
    
    elif x.strip() in ['Motor Vehicle - Comp',
                'Motor Vehicle - TPPD']:
        return 'IAL Motor'
    
    elif x.strip() in ['All Credit',
                'Alliance Leasing',
                'Automotive Lease Packaging Protect',
                'Autovate',
                'Easi RideShare',
                'Easifleet Lumley',
                'Easifleet',
              'Fleetcare Commercial',
              'Fleetcare Plus',
              'Fleetpro Plus',
              'Paradigm Gold',
              'Paradigm Budget',
              'Fleetpro Plus',
              'Fleetcare Commercial',
              'Prorisk Commercial Fleet',
              'Selectus InsureWise',
              'Selectus InsureWise Plus',
              'Stratton Finance',
              'TEC Finance']:
        return 'Inter Hannover'
    
    elif x.strip() in ['IHAB Argis - Private Motor',
                'IHAB Argis - Farm Motor']:
        return 'Inter Hannover (Argis)'
    
    elif x.strip() in ['Mansions - Building and Contents',
                'Mansions Plus Motor']:
        return 'Mansions'
    
    elif x.strip() in ['One Underwriting Specialty Motor Insurance',
                'Securus Motor Vehicle Insurance']:
        return 'One Underwriting Specialty Motor Insurance'
    
    elif x.strip().startswith('Progressive'):
        return 'Progressive'
    
    elif x.strip() in ['QBE Insurance Box','Insurance Box']:
        return 'QBE Insurance Box'
    
    elif x.strip() in ['Famous Bike Insurance',
               'Famous Car Insurance',
               'Famous Dirt Bike Insurance',
               'RAC Classic Car Insurance',
               'RACQ Enthusiast Car Insurance',
               'RACQ Enthusiast Motorcycle Insurance']:
        return 'RACQ'
    
    else:
        return x
    
        


In [7]:
def createDivision(x):
    
    if x in ['Allianz', 
    'Allied World',
    'Australian Reliance',
    'Calliden Insurance Limited',
    'CIL Motor',
    'Dawes',
    'Famous Motor',
    'Famous Car Insurance',
    'Famous Car Insurance ',
    'GLA Argis - Farm Motor',
    'GLA Australian Reliance',
    'GLA Dawes Bike',
    'GLA Dawes Motor',
    'GLA Famous',
    'IAL Motor',
    'Inter Hannover',
    'Inter Hannover (Argis)',
    'NLC',
    'Progressive',
    'Prorisk',
    'QBE Insurance Box',
    'RAC Classic',
    'RACQ',
    'Melbourne Motor - Other',
    'Automotive Lease Packaging Plus',
    'Securus Motor Vehicle Insurance', 
    'One Underwriting Specialty Motor Insurance',
    'Sydney Motor - Other',
            'error',
            'Motor',
            'Register in Error'] or x.startswith('AZ -'):
        return 'Motor'
    
    elif x in ['BH IAL Property', 
                'BH Landlords', 
                'Chubb Insurance', 
                'Chubb Landlords Claims', 
                'Chubb Mansions', 
                'CIL - Domestic', 
                'IAL Property', 
                'Mansions', 
                'QUS (excl Liability)',
                'Watkins Taylor Stone - Landlords', 
    'Melbourne Property - Other']:
        return 'Property'
    
    elif x in ['Calliden Warranty', 
                'CIL - Builders Warranty', 
                'CIL - Builders Warranty VIC', 
                'CIL - Builders Warranty WA', 
                'CIL Domestic',
                'CIL - SUA', 
                'GLA - SUA', 
                'NLC Lease Guarantee', 
                'NSW Warranty', 
                'Owner Builder',
                'Calliden - SUA']:
        return 'Other'
    
    return 'error'
    

In [8]:
def createIncidentDate(x):
    if x == '6147' or x =='Perth' or x =='WA':
        return 'WA'
    
    elif x == 'coomNSW' or x =='NSW':
        return 'NSW'
    
    elif x == 'QLD' or x =='Qld':
        return 'QLD'
    
    elif x == 'SA':
        return 'SA'
    
    elif x == 'TAS' or x=='TASMANIA':
        return 'TAS'

    elif x == 'VIC' or x =='Vic':
        return 'VIC'
    
    elif x == 'NT':
        return 'NT'
    
    elif x == 'ACT':
        return 'ACT'
    
    elif x == 'UNKNOWN'or x =='&nbsp;'or x == 'O/S':
        return ''
    return 'error'

In [68]:
CFT_df_union_copy = CFT_df_union.copy()

In [69]:
import datetime as dt
from datetime import datetime 
import numpy as np

# Drop columns

CFT_df_union_copy = CFT_df_union_copy.loc[:,['Claim No','Loss State',
       'Loss Postcode', 'Loss Date', 'Loss Type','Registered Date',
       'Settlement Date','Catastrophe','Policy Type','Insurer Name','Vehicle Classification', 'Vehicle Type',
       'Vehicle Make', 'Vehicle Model', 'Vehicle Registration',
       'Vehicle Year', 'Driver Lastname', 'Driver Firstname',
       'Driver DOB', 'Driver Code', 'Excess Fee', 'Total Excess Charged',
       'Total Excess Outstanding', 'Total Excess Received', 'Total Loss']]
# replace Policy Type
CFT_df_union_copy['Policy Type'] = CFT_df_union_copy['Policy Type'].apply(lambda x: replacePolicyType(x))

# create Division
CFT_df_union_copy['Division'] =  CFT_df_union_copy['Policy Type'].apply(lambda x: createDivision(x))

In [70]:
# Convert String to datetime
CFT_df_union_copy['Registered Date'] = CFT_df_union_copy['Registered Date'].fillna('').apply(lambda x: x if x == '' else datetime.strptime(x,'%d/%m/%Y') )
CFT_df_union_copy['Settlement Date'] = CFT_df_union_copy['Settlement Date'].fillna('').apply(lambda x: x if x == '' else datetime.strptime(x,'%d/%m/%Y'))
CFT_df_union_copy['Loss Date']      =   CFT_df_union_copy['Loss Date'].fillna('').apply(lambda x: x if x == '' else datetime.strptime(x,'%d/%m/%Y'))


# Filter Relative Date (last 26 months)
CFT_df_union_copy = CFT_df_union_copy[CFT_df_union_copy['Registered Date'] >= dt.date.today() - pd.offsets.MonthBegin(26)]
# Filter Claim No
CFT_df_union_copy = CFT_df_union_copy[CFT_df_union_copy['Claim No'] != 'SLYP009176']

#Create Status
CFT_df_union_copy['Status'] = CFT_df_union_copy['Settlement Date'].apply(lambda x:'OPEN' if x =='' else 'CLOSE')

#Create Incident State
CFT_df_union_copy['IncidentState'] = CFT_df_union_copy['Loss State'].apply(lambda x:createIncidentDate(x))

# Rename the column
CFT_df_union_copy = CFT_df_union_copy.rename(columns={'Policy Type':'Client'})

In [71]:
CFT_df_union_copy.columns.values

array(['Claim No', 'Loss State', 'Loss Postcode', 'Loss Date',
       'Loss Type', 'Registered Date', 'Settlement Date', 'Catastrophe',
       'Client', 'Insurer Name', 'Vehicle Classification', 'Vehicle Type',
       'Vehicle Make', 'Vehicle Model', 'Vehicle Registration',
       'Vehicle Year', 'Driver Lastname', 'Driver Firstname',
       'Driver DOB', 'Driver Code', 'Excess Fee', 'Total Excess Charged',
       'Total Excess Outstanding', 'Total Excess Received', 'Total Loss',
       'Division', 'Status', 'IncidentState'], dtype=object)

In [72]:
CFT_df_union_copy['Table Names']  = 'CFT'

## FigTree


In [73]:
## FigTree (excel)
figTree_df = pd.read_excel(path + 'figtree.xlsx',encoding = 'utf-8')

In [74]:
def groupClient(x):
    if x == 'FA' or x == 'SG Fleet':
        return 'SG Fleet'
    elif x == 'MAIN' or x == 'WW Direct':
        return 'WW Direct'
    return x

def createIncidentState(x):
    if x in ['QLD','VIC','NSW','WA','TAS','NT','ACT','SA']:
        return x
    if float(x) < 800: 
        return 'ACT'
    elif float(x) < 1000: 
        return 'NT'
    elif float(x) < 2600: 
        return 'NSW'
    elif float(x) <= 2620:
         return 'ACT'
    elif float(x) < 2900: 
        return 'NSW'
    elif float(x) <= 2914: 
        return 'ACT'
    elif float(x) < 3000: 
        return 'NSW'
    elif float(x) < 4000: 
        return 'VIC'
    elif float(x)< 5000:
        return 'QLD'
    elif float(x) < 6000: 
        return 'SA'
    elif float(x) < 7000: 
        return 'WA'
    elif float(x) < 8000: 
        return 'TAS'
    elif float(x) < 9000: 
        return 'VIC'
    elif float(x) < 10000: 
        return 'QLD'
    return 'Unknown'


In [75]:
figTree_df_copy = figTree_df.copy()

In [76]:
# Rename field
figTree_df_copy['Status'] = figTree_df_copy['Finalised'].apply(lambda x: 'OPEN' if pd.isna(x) else 'CLOSE')
figTree_df_copy['IncidentState'] = figTree_df_copy['Repairer PCode'].apply(lambda x: createIncidentState(x))
figTree_df_copy['Division'] = 'WW'
figTree_df_copy['Client'] = figTree_df_copy['Client'].apply(lambda x:groupClient(x))

figTree_df_copy = figTree_df_copy.rename(columns={'Input Date':'Registered Date','IG REF':'Claim No','Loss Date':'Acc Date','Repairer PCode':'Loss Postcode'})
figTree_df_copy.columns.values
figTree_output_column = ['IncidentState','Status','Division','Client','Claim No','Registered Date','Loss Date','Repairer Name','Loss Postcode','InsClNo']

figTree_df_copy = figTree_df_copy.loc[:,figTree_output_column]
figTree_df_copy['Table Names']  = 'FigTree'

In [77]:
figTree_df_copy.columns.values

array(['IncidentState', 'Status', 'Division', 'Client', 'Claim No',
       'Registered Date', 'Loss Date', 'Repairer Name', 'Loss Postcode',
       'InsClNo', 'Table Names'], dtype=object)

## Fast Track

In [78]:
def createIncidentState_FT(x):
    import re
    x = str(x)
    if re.match('6147',x):
        return  'WA'
    elif re.match('6167',x):
        return  'WA'
    elif x =='coomNSW':
        return  'NSW'
    elif x.lower() =='nsw':
        return  'NSW'
    elif x =='Perth':
        return  'WA'
    elif x.lower() =='qld':
        return  'QLD'
    elif x =='Q;D':
        return  'QLD'
    elif x.lower() == 'sa':
        return  'SA'
    elif x.lower() =='nt':
        return  'NT'
    elif x.lower() == 'tas':
        return  'TAS'
    elif x =='TASMANIA':
        return  'TAS'
    elif x.lower() =='vic':
        return  'VIC'
    elif x =='Vic':
        return  'VIC'
    elif x =='Victoria':
        return  'VIC'
    elif x =='VICTORIA':
        return  'VIC'
    elif x.lower() =='act':
        return  'ACT'
    elif x.lower() == 'wa':
        return  'WA'
    elif x =='Western Australia':
        return  'WA'
    elif x =='WESTERN AUSTRALIA':
        return  'WA'
    elif x =='&nspb;':
        return  'nan1'
    elif x =='UNKNOWN':
        return  ''
    elif pd.isna(x):
        return  ''
    return ''

In [115]:
## Fast Track Weekly (excel)
FT_df = pd.read_excel(path + 'Fast Track Data weekly extract.xlsx')

In [141]:
FT_df_copy = FT_df.copy()

In [142]:
FT_df_copy.columns.values

array(['Consolidator', 'Client', 'Claim Number', 'Report Date', 'Status',
       'Tech Sign Off', 'Closed Date', 'Last Reopen Date',
       'Reopen Reason', 'Excess', 'SentTo Insurer', 'Cost Centre 1',
       'Cost Centre 2', 'Business Unit', 'PIN', 'Operating Arm',
       'Department', 'Catastrophe Date', 'Incident Date',
       'Incident State', 'Incident PostCode', 'Incident Code', 'Fault',
       'Purpose of Trip', 'Vehicle Registration', 'Vehicle Make',
       'Vehicle Model', 'Lease Type', 'Drivable', 'Driver First Name',
       'Driver Last Name', 'Insurer Cost', 'Recovered',
       'Recovered inc tax', 'Client Cost', 'All Payments',
       'All Payments inc tax', 'Client Outstanding', 'TP Involvement',
       'Injuries', 'Total Loss', 'Client Repairer',
       'Repairs Authorised inc tax', 'Repairs Invoiced inc tax',
       'Estimate inc tax', 'Client Vehicle Assessor', 'Assessed inc tax',
       'LOD Sent', 'Initial LOD Date', 'Demand Amount', 'Recovery Status'],
      dtype=o

In [143]:
FT_df_copy['Division']  = 'Fleet'
FT_df_copy['IncidentState']  = FT_df_copy['Incident State'].apply(lambda x:createIncidentState_FT(x))


FT_df_copy = FT_df_copy.rename(columns = {'Client':'Client Detail','Consolidator':'Client','Claim Number':'Claim No','Incident PostCode':'Loss Postcode'})
FT_df_copy_output_column = ['IncidentState','Division','Client','Client Detail','Claim No','Report Date','Status','Catastrophe Date','Incident Date','Incident State','Loss Postcode','Incident Code']
FT_df_copy = FT_df_copy.loc[:,FT_df_copy_output_column]
FT_df_copy['Table Names']  = 'FT'

In [144]:
FT_df_copy

,IncidentState,Division,Client,Client Detail,Claim No,Report Date,Status,Catastrophe Date,Incident Date,Incident State,Loss Postcode,Incident Code,Table Names
0,WA,Fleet,SG Fleet Australia Pty Limited,Department of Defence White Fleet,403238,2017-02-17,CLOSE,NaT,2017-02-16 00:00:00,WA,6056,Z1L - Windscreen,FT
1,VIC,Fleet,Australia Post,Australia Post,403532,2017-02-20,CLOSE,NaT,2017-02-17 00:00:00,VIC,3000,C2R - TP Hits Client In Rear,FT
2,SA,Fleet,Lease Plan Total Cover,Wurth Australia Pty Ltd,403446,2017-02-20,CLOSE,NaT,2017-02-19 00:00:00,SA,5108,C2R - TP Hits Client In Rear,FT
3,WA,Fleet,SG Fleet Australia Pty Limited,Department of Defence White Fleet,403227,2017-02-17,CLOSE,NaT,2017-02-11 00:00:00,WA,6084,Z1L - Windscreen,FT
4,NSW,Fleet,SG Fleet Australia Pty Limited,Department of Defence White Fleet,403230,2017-02-17,CLOSE,NaT,2016-11-19 00:00:00,NSW,2011,B1L - Client Reversing,FT
5,QLD,Fleet,SG Fleet Australia Pty Limited,Department of Defence White Fleet,403241,2017-02-17,CLOSE,NaT,2017-02-11 00:00:00,QLD,4306,W1L - Hail,FT
6,NSW,Fleet,SG Fleet Australia Pty Limited,Department of Defence White Fleet,403505,2017-02-20,CLOSE,NaT,2017-02-13 00:00:00,NSW,2549,S4L - Hit Animal,FT
7,ACT,Fleet,SG Fleet Australia Pty Limited,Department of Defence White Fleet,403543,2017-02-20,CLOSE,NaT,2017-02-20 00:00:00,ACT,2601,B1L - Client Reversing,FT
8,QLD,Fleet,AB Group,Avis Australia,403314,2017-02-18,CLOSE,NaT,2015-02-08 00:00:00,QLD,4630,A3R - Breach of Contract,FT
9,WA,Fleet,SG Fleet Australia Pty Limited,Department of Defence White Fleet,403559,2017-02-20,CLOSE,NaT,2016-10-20 00:00:00,WA,6010,B1L - Client Reversing,FT


## Union 1 (Union Three sources)

In [145]:
unionAll_df = pd.concat([CFT_df_union_copy,figTree_df_copy,FT_df_copy],ignore_index=True)

C:\Users\LiaoG\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [153]:
unionAll_df.groupby('Table Names')['Loss Date'].count()

Table Names
CFT        23051
FT             0
FigTree        0
Name: Loss Date, dtype: int64

In [99]:
unionAll_df.groupby('Table Names')['InsClNo'].count()

Table Names
CFT            0
FT             0
FigTree    17534
Name: InsClNo, dtype: int64